In [1]:
%matplotlib inline

In [2]:
import pandas as pd

In [18]:
df = (pd.read_csv("../input/h_2_courses.csv")
        .rename(columns={"наименование": "name", "max-score, максим который набрать": "max_possible_score", 
                         "проходной ли балл?": "is_passed", "сам ли назначил?": "does_he_appointed",
                         "когда активирован курс": "course_activated_at", "когда завершен": "course_finished_at",
                         "в срок ли завершен": "completed_in_period"
                        })
        .drop(['Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 
               'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'], 1)
     )

In [15]:
print([x for x in df.columns])

['uid', 'name', 'max_possible_score', 'score', '%', 'is_passed', 'does_he_appointed', 'course_activated_at', 'course_finished_at', 'completed_in_period', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20']


In [19]:
df.head()

,uid,name,max_possible_score,score,%,is_passed,does_he_appointed,course_activated_at,course_finished_at,completed_in_period
0,u995,Служба обслуживания сотрудников,100,100.00,100.00,1,0,21.10.2014,24.10.2014,1.0
1,u995,"Обучающий курс: ""Эффективная работа с кросс-ма...",100,100.00,100.00,1,0,08.10.2014,13.10.2014,1.0
2,u995,Технология продаж кухонной техники,100,100.00,100.00,1,1,01.10.2014,01.10.2014,NaN
3,u995,Технология продаж кухонной техники,100,98.75,98.75,1,1,27.09.2014,27.09.2014,NaN
4,u995,Маркетинговая акция осени,100,100.00,100.00,1,0,22.09.2014,26.09.2014,1.0


In [5]:
df.columns

Index(['uid', 'наименование', 'max-score, максим который набрать', 'score',
       '%', 'проходной ли балл?', 'сам ли назначил?', 'когда активирован курс',
       'когда завершен', 'в срок ли завершен', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20'],
      dtype='object')